In [18]:
import sqlite3, string
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

from imblearn.pipeline import Pipeline
import joblib

In [19]:
con = sqlite3.connect('prods.db')
products = pd.read_sql("SELECT * FROM products", con)

products.drop_duplicates(inplace=True)
products.dropna(inplace=True)

products['category_name'] = products['category_name'].str.lower()

products.head()

,id,shop,product_id,name,code,category_name,category_code,price
0,1,perekrestok,32392,"Голубика Маркет Fresh, 125г",golubika-market-fresh-125g,готовимся к праздникам,gotovimsa-k-prazdnikam,239.99
1,2,perekrestok,10299,"Горошек зелёный Маркет Перекрёсток, 400г",gorosek-zelenyj-market-perekrestok-400g,готовимся к праздникам,gotovimsa-k-prazdnikam,93.99
2,3,perekrestok,404062,Колбаса Докторская варёная категории А в натур...,kolbasa-doktorskaa-varenaa-kategorii-a-v-natur...,готовимся к праздникам,gotovimsa-k-prazdnikam,779.99
3,4,perekrestok,34160,Клементины,klementiny,готовимся к праздникам,gotovimsa-k-prazdnikam,229.99
4,5,perekrestok,32227,Бананы,banany,готовимся к праздникам,gotovimsa-k-prazdnikam,119.99


In [20]:
products.boxplot(column=['price'])
products.sort_values(by='price', ascending=False).head(15)

,id,shop,product_id,name,code,category_name,category_code,price
6136,6137,ashan,13403940,"Бренди Torres, Jaime I Испания, 0,7 л + 2 Стакана",torres-30-hayme-i-0-70,алкоголь (самовывоз),alkogol,11999.95
7751,7752,ashan,16016800,"Коньяк «Коктебель» 25 лет Россия, 0,7 л",koktebel-staryy-25-let-0-7,алкоголь (самовывоз),alkogol,10768.99
7679,7680,ashan,6063235,Коньяк Lheraud Vieux Millenaire в подарочной у...,konyak_lero_ve_milliner_07l_43,алкоголь (самовывоз),alkogol,7999.00
7690,7691,ashan,6062554,"Коньяк Remy Martin VSOP Франция, 0,7 л",remi_martin_vsop_konyak07lpu_40,алкоголь (самовывоз),alkogol,7067.99
7673,7674,ashan,8746843,Коньяк Hine Rare в подарочной упаковке Франция...,konyak_hayn_pap_vsop_07l_40,алкоголь (самовывоз),alkogol,6999.00
7648,7649,ashan,10912670,Коньяк ARARAT Двин в подарочной упаковке Армен...,konyak-ararat-dvin-p-u-0-7,алкоголь (самовывоз),alkogol,6947.99
6079,6080,ashan,12801143,"Аперитив Aperol Aperitivo Италия, 3 л",spirt-napitok-aperol-11-3l,алкоголь (самовывоз),alkogol,6499.95
7647,7648,ashan,15701531,"Коньяк ARARAT Васпуракан Армения, 0,7 л",konyak-ararat-vaspurakan-0-7,алкоголь (самовывоз),alkogol,6092.99
423,424,perekrestok,40091,Икра нерки зернистая,ikra-nerki-zernistaa,готовимся к праздникам,gotovimsa-k-prazdnikam,5999.00
7656,7657,ashan,11682659,Коньяк Camus V. S. O. P в подарочной упаковке ...,konyak-kamyu-vsop-0-7-p-u-40,алкоголь (самовывоз),alkogol,5998.99


In [21]:
products['price'].hist(bins=50)
products['price'].describe()

count    21769.000000
mean       261.487882
std        412.358708
min          1.590000
25%         74.990000
50%        139.900000
75%        279.000000
max      11999.950000
Name: price, dtype: float64

In [22]:
products['category_name'].value_counts().plot.bar()

<Axes: title={'center': 'ТЕПЛОВАЯ КАРТА МОДЫ СТОИМОСТИ ТОВАРОВ'}, xlabel='category_name', ylabel='Категория'>

In [36]:
products['general_category'] = np.nan

def general_category(row):
    if 'молоко' in row['category_name'] or 'молочные' in row['category_name'] or 'сыр' in row['category_name'] or 'яйца' in row['category_name']:
        return 'Молочные продукты'
    elif 'мясные' in row['category_name'] or 'колбасные' in row['category_name']  or 'мясо' in row['category_name'] or 'сосиски' in row['category_name']:
        return 'Мясные продукты'
    elif 'рыба' in row['category_name']:
        return 'Рыба и морепродукты'
    elif 'овощи' in row['category_name'] or 'фрукты' in row['category_name'] or 'орехи' in row['category_name'] or 'снеки' in row['category_name']:
        return 'Овощи, фрукты, закуски'
    elif 'бакалея' in row['category_name'] or 'соусы' in row['category_name'] or 'макароны' in row['category_name']:
        return 'Бакалея и соусы'
    elif 'сладости' in row['category_name']:
        return 'Сладости'
    elif 'консервы' in row['category_name'] or 'мед' in row['category_name'] or 'варенье' in row['category_name']:
        return 'Консервы, мед, варенье'
    elif 'выпечка' in row['category_name'] or 'хлеб' in row['category_name']:
        return 'Выпечка и хлеб'
    elif 'чай' in row['category_name'] or 'кофе' in row['category_name'] or 'сахар' in row['category_name']:
        return 'Чай, кофе, какао, сахар'
    elif 'напитки' in row['category_name'] or 'алкоголь' in row['category_name']:
        return 'Напитки'
    else:
        return 'Прочее'

products['general_category'] = products.apply(general_category, axis=1)
products.general_category = products.general_category.astype('category')
products.sample(20)

,id,shop,product_id,name,code,category_name,category_code,price,general_category,clear_category
5081,5082,magnit,1000049309,Персики Premiere of Taste 720мл,persiki-premiere-of-taste-720ml,"консервы, мёд, варенье",konservy-myod-varenye,249.99,"Консервы, мед, варенье",консервы мёд варенье
12899,12900,ashan,16463915,"Батончик te Gusto из абрикоса, 25 г",batonchik-iz-abrik-te-gusto-25g,"чипсы, снеки, сухофрукты",orehi-suhofrukty-sneki,60.99,"Овощи, фрукты, закуски",чипсы снеки сухофрукты
4885,4886,magnit,3070960501,Драже Mentos Фруктовый 37.5г,drazhe-mentos-fruktovy-375g,"сладости, торты, пирожные",sladosti-torty-pirozhnye,65.99,Сладости,сладости торты пирожные
7534,7535,ashan,15896668,Игристое вино Sandiliano белое полусладкое Ита...,igr-vi-sandilyano-bl-psl-0-75l,алкоголь (самовывоз),alkogol,749.99,Напитки,алкоголь самовывоз
13723,13724,ashan,8342154,"Чебупели «Горячая штучка» Сочные с мясом, 300 г",chebupeli_svingov_03kg,замороженные продукты,zamorozhennye-produkty,162.99,Прочее,замороженные продукты
929,930,perekrestok,407663,Сахар тростниковый кристаллический нерафиниров...,sahar-trostnikovyj-kristalliceskij-nerafinirov...,от перекрёстка,ot-perekrestka,239.99,Прочее,от перекрёстка
1504,1505,perekrestok,19173,Масло оливковое Monini Classico нерафинированн...,maslo-olivkovoe-monini-classico-nerafinirovann...,"макароны, крупы, масло, специи",makarony-krupy-maslo-specii,719.99,Бакалея и соусы,макароны крупы масло специи
6075,6076,ashan,13106429,"Puente De Piedra красное сухое Испания, 0,75 л",puente-de-pedra-karinena,алкоголь (самовывоз),alkogol,698.96,Напитки,алкоголь самовывоз
381,382,perekrestok,357005,Набор Hilltop Коллекционные шедевры чай черный...,nabor-hilltop-kollekcionnye-sedevry-caj-cernyj...,готовимся к праздникам,gotovimsa-k-prazdnikam,779.99,Прочее,готовимся к праздникам
7466,7467,ashan,7124086,Игристое вино Fanagoria розовое полусладкое Ро...,vino_igr_fanagor_psl_roz_075,алкоголь (самовывоз),alkogol,349.00,Напитки,алкоголь самовывоз


In [24]:
products.category_name = products.category_name.astype('category')
products.category_code = products.category_code.astype('category')
products.general_category = products.general_category.astype('category')
products.dtypes

id                     int64
shop                  object
product_id             int64
name                  object
code                  object
category_name       category
category_code       category
price                float64
general_category    category
dtype: object

In [25]:
def preprocess_text(input_string):
    clear_string = input_string.translate(str.maketrans('', '', string.punctuation + string.digits))
    clear_string = clear_string.lower()
    return clear_string

products['clear_category']=products['category_name'].apply(preprocess_text)
products.sample(10)

,id,shop,product_id,name,code,category_name,category_code,price,general_category,clear_category
20859,20860,ashan,903817,Напиток энергетический АШАН Красная птица гази...,napitok-energeticheskiy-auchan-krasnaya-ptica-...,"вода, соки, напитки",voda-soki-napitki,43.99,Напитки,вода соки напитки
5435,5436,magnit,1000141716,Хлеб Хлебный Дом Геркулес зерновой нарезка 250г,khleb-khlebny-dom-gerkules-zernovoy-narezka-250g,"хлеб, выпечка",khleb-vypechka,76.99,Выпечка и хлеб,хлеб выпечка
11293,11294,ashan,8341390,Окорок варено-копченый «Ближние Горки» По-тамб...,okorok_po_tambovski_350g_bg,колбасные изделия,kolbasnye-izdeliya,149.95,Мясные продукты,колбасные изделия
9509,9510,ashan,211263,"Молоко питьевое «Искренне Ваш» 3,2% БЗМЖ, 930 мл",moloko-iskrenne-vash-3-2-930-g,"молочные продукты, яйца",moloko-syr-yayca,85.99,Молочные продукты,молочные продукты яйца
19185,19186,ashan,17788953,"Приправа для супа Orient, 14 г",priprava-d-supa-14gr-orient,бакалея,bakaleya,49.99,Бакалея и соусы,бакалея
11762,11763,ashan,132877,"Конина тушеная «Халяль», 325 г",konina-tushenaya-halyal-325-g,колбасные изделия,kolbasnye-izdeliya,230.99,Мясные продукты,колбасные изделия
4249,4250,magnit,1000276881,Приправа Индана для шашлыка 15г,priprava-indana-dlya-shashlyka-15g,"бакалея, соусы",bakaleya-sousy,40.99,Бакалея и соусы,бакалея соусы
14239,14240,ashan,18449218,"Мидии «Каждый день» отварные в масле, 200 г",kd-midii-v-masle-200g-1,"рыба, икра, морепродукты",ryba-ikra-moreprodukty,110.99,Рыба и морепродукты,рыба икра морепродукты
12787,12788,ashan,14320574,"Фисташки обжаренные, 200 г",fistashki-zh-s-p-p-200g,"овощи, фрукты, орехи",ovoschi-frukty-zelen-griby-yagody,339.99,"Овощи, фрукты, закуски",овощи фрукты орехи
7007,7008,ashan,6063081,"Виски Scotch Terrier Россия, 0,7 л",viski_ssotchterrrier_07,алкоголь (самовывоз),alkogol,1024.99,Напитки,алкоголь самовывоз


In [26]:
x = products.clear_category
y = products.general_category
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(X_train, y_train)
y_pred = text_clf.predict(X_test)

print('Score:', text_clf.score(X_test, y_test))

Score: 0.9991648106904232


In [27]:
joblib.dump(text_clf, 'model.pkl')

['model.pkl']

In [28]:
c = sqlite3.connect('prods.db')
pr = pd.read_sql("SELECT * FROM products", c)

pr.sample(10)

,id,shop,product_id,name,code,category_name,category_code,price
13017,13018,ashan,15614542,Луковые кольца KDV со вкусом аргентинских ребр...,lukovye-kolca-200-g-6-argen,"Чипсы, снеки, сухофрукты",orehi-suhofrukty-sneki,130.99
17288,17289,ashan,101734,"Хрен сливочный АШАН Красная птица, 180 г",hren-auchan-krasnaya-ptica-slivochnyy-180-g,Бакалея,bakaleya,89.99
8826,8827,ashan,103646,Масло сладкосливочное «Милково» Крестьянское н...,maslo-sladko-slivochnoe-milkovo-nesolenoe-kres...,"Молочные продукты, яйца",moloko-syr-yayca,369.00
8979,8980,ashan,17378549,Йогурт питьевой АШАН Красная птица со злаками ...,kp-yogurt-pit-zlaki-2-5-750g,"Молочные продукты, яйца",moloko-syr-yayca,119.99
15945,15946,ashan,12179276,Пирожные миндальные «Акульчев» Макаронc со вку...,macaron-so-vkus-cherniki-48g,Сладости,konditerskie_izdeliya,122.99
376,377,perekrestok,411153,Шоколадные фигурки Золотое Правило Пряничный д...,sokoladnye-figurki-zolotoe-pravilo-pranicnyj-d...,Готовимся к праздникам,gotovimsa-k-prazdnikam,239.99
4,5,perekrestok,32227,Бананы,banany,Готовимся к праздникам,gotovimsa-k-prazdnikam,119.99
9285,9286,ashan,10514203,"Яйца куриные «Золото Сеймы» деревенские С1, 10 шт",yayco-derevenskoe-s1-10-sht-ef,"Молочные продукты, яйца",moloko-syr-yayca,99.49
12612,12613,ashan,130546,"Вишня «Белкендорф» вяленая, 150 г",vishnya-belkendorf-vyalenaya-150-g,"Овощи, фрукты, орехи",ovoschi-frukty-zelen-griby-yagody,173.99
19397,19398,ashan,225005,Смесь для выпечки «С.Пудовъ» Кекс в кружке лим...,smes-dlya-vypechki-s-pudov-keks-v-kruzhke-limo...,Бакалея,bakaleya,39.90


In [29]:
with open('model.pkl', 'rb') as f:
    clf2 = joblib.load(f)

def preprocess_text(input_string):
    clear_string = input_string.translate(str.maketrans('', '', string.punctuation + string.digits))
    clear_string = clear_string.lower()
    return clear_string

pr['clear_category'] = pr['category_name'].apply(preprocess_text)

prediction = clf2.predict(pr['clear_category'])
pr['general_category'] = prediction

pr.sample(10)

,id,shop,product_id,name,code,category_name,category_code,price,clear_category,general_category
2192,2193,perekrestok,48431,Напиток безалкогольный Черноголовка Дюшес гази...,napitok-bezalkogolnyj-cernogolovka-duses-gazir...,"Соки, воды, напитки",soki-vody-napitki,71.99,соки воды напитки,Напитки
21191,21192,ashan,249663,"Нектар «Любимый» Мультифрукт, 1,93 л",nektar-lyubimyy-multifrukt-1-93-l,"Вода, соки, напитки",voda-soki-napitki,179.90,вода соки напитки,Напитки
15570,15571,ashan,16601756,Печенье сахарное Falcone Кантуччи с фисташками...,pech-kantuchchi-s-fist-i-lim-180g,Сладости,konditerskie_izdeliya,349.00,сладости,Сладости
20832,20833,ashan,14857369,"Какао-порошок Sole, 150 г",kakao-poroshok-150g-bio,"Чай, кофе, горячие напитки",chay-kofe-sladosti,549.99,чай кофе горячие напитки,"Чай, кофе, какао, сахар"
18927,18928,ashan,101758,"Перец сладкий «Дядя Ваня» маринованный, 680 г",perec-sladkiy-dyadya-vanya-marinovannyy-680-g,Бакалея,bakaleya,228.99,бакалея,Бакалея и соусы
20892,20893,ashan,2396679,Напиток сильногазированный АШАН Красная птица ...,napitok-silnogazirovannyy-auchan-krasnaya-ptic...,"Вода, соки, напитки",voda-soki-napitki,88.99,вода соки напитки,Напитки
1488,1489,perekrestok,21339,"Лист лавровый Kotanyi отборный, 5г",list-lavrovyj-kotanyi-otbornyj-5g,"Макароны, крупы, масло, специи",makarony-krupy-maslo-specii,65.99,макароны крупы масло специи,Бакалея и соусы
7063,7064,ashan,10912687,"Водка Danzka Германия, 0,7 л",vodka-danska-0-7,Алкоголь (самовывоз),alkogol,1199.00,алкоголь самовывоз,Напитки
13110,13111,ashan,10870655,"Снеки кукурузные Cheetos Кетчуп, 85 г",cheetos-ketchup-85g,"Чипсы, снеки, сухофрукты",orehi-suhofrukty-sneki,76.99,чипсы снеки сухофрукты,"Овощи, фрукты, закуски"
1760,1761,perekrestok,42003,"Жевательная резинка Dirol Маракуйя без сахара,...",zevatelnaa-rezinka-dirol-marakuja-bez-sahara-1...,"Шоколад, конфеты, сладости",sokolad-konfety-sladosti,41.99,шоколад конфеты сладости,Сладости


In [30]:
import matplotlib.pyplot as plt
import seaborn as sns

top_10 = pr.sort_values(by='price', ascending=False).head(10)
sns.set_style('darkgrid')
plt.figure(figsize=(7,5))
sns.barplot(x = 'price', y = 'name', data = top_10, palette='rocket')
plt.title('10 САМЫХ ДОРОГИХ ПРОДУКТОВ', fontsize=22)
plt.xlabel('Стоимость', fontsize=16)
plt.ylabel('Наименование', fontsize=16)
plt.tick_params(axis='x', labelsize=10)
plt.tick_params(axis='y', labelsize=10)
#plt.savefig('top_10_max.png', bbox_inches = 'tight')

C:\Users\shany\AppData\Local\Temp\ipykernel_12052\3764666843.py:7: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x = 'price', y = 'name', data = top_10, palette='rocket')


In [31]:
top_10 = pr.sort_values(by='price', ascending=True).head(10)
sns.set_style('darkgrid')
plt.figure(figsize=(7,5))
sns.barplot(x = 'price', y = 'name', data = top_10, palette='rocket')
plt.title('10 САМЫХ ДЕШЕВЫХ ПРОДУКТОВ', fontsize=22)
plt.xlabel('Стоимость', fontsize=16)
plt.ylabel('Наименование', fontsize=16)
plt.tick_params(axis='x', labelsize=10)
plt.tick_params(axis='y', labelsize=10)

C:\Users\shany\AppData\Local\Temp\ipykernel_12052\1191127760.py:4: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x = 'price', y = 'name', data = top_10, palette='rocket')


In [32]:
conditions = [(pr['shop'] == 'ashan'), (pr['shop'] == 'magnit'), (pr['shop'] == 'perekrestok')]
values = ['Ашан', 'Магнит', 'Перекресток']
pr['shop_rus'] = np.select(conditions, values)
pr.shop_rus = pr.shop_rus.astype('category')

plt.figure(figsize=(10,5))
pivot_table = pr.pivot_table(index='general_category', columns='shop_rus', values='price', aggfunc='mean')
plt.title('ТЕПЛОВАЯ КАРТА СРЕДНЕЙ СТОИМОСТИ ТОВАРОВ', fontsize=20)
sns.heatmap(pivot_table, cmap='rocket', annot=True, fmt=".1f")
plt.xlabel('Магазин', fontsize=16)
plt.ylabel('Категория', fontsize=16)
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)
#plt.show()

In [33]:
plt.figure(figsize=(10,5))
pivot_table = pr.pivot_table(index='general_category', columns='shop_rus', values='price', aggfunc=lambda x: x.mode().max())
plt.title('ТЕПЛОВАЯ КАРТА МОДЫ СТОИМОСТИ ТОВАРОВ', fontsize=20)
sns.heatmap(pivot_table, cmap='rocket', annot=True, fmt=".1f")
plt.xlabel('Магазин', fontsize=16)
plt.ylabel('Категория', fontsize=16)
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)